# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,81.086317,0.060163,1234,81.146481,81.026154,162.292962,162.052308
6,185.882599,0.911164,1234,186.793763,184.971435,373.587526,369.942869
7,230.589537,0.987688,1234,231.577225,229.601849,463.154451,459.203698
8,912.852645,-0.676317,1234,912.176329,913.528962,1824.352658,1827.057924
9,545.342129,0.560034,1234,545.902163,544.782095,1091.804327,1089.564189
10,464.497619,-0.586023,1234,463.911595,465.083642,927.823191,930.167283
11,302.310023,0.380166,1234,302.690189,301.929856,605.380378,603.859713
12,556.357496,0.883265,1234,557.240761,555.474230,1114.481522,1110.948461
13,135.260856,0.369487,1234,135.630343,134.891370,271.260686,269.782739
14,190.879967,-0.599899,1234,190.280068,191.479865,380.560137,382.959731


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-81.146481,81.026154
6,-186.793763,184.971435
7,-231.577225,229.601849
8,-912.176329,913.528962
9,-545.902163,544.782095
10,-463.911595,465.083642
11,-302.690189,301.929856
12,-557.240761,555.474230
13,-135.630343,134.891370
14,-190.280068,191.479865


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-81.146481,81.026154
6,-186.793763,184.971435
7,-231.577225,229.601849
8,-912.176329,913.528962
9,-545.902163,544.782095
10,-463.911595,465.083642
11,-302.690189,301.929856
12,-557.240761,555.474230
13,-135.630343,134.891370
14,-190.280068,191.479865


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[ -81.14648077,   81.02615389],
       [-186.79376304,  184.97143458],
       [-231.57722532,  229.60184877],
       [-912.17632884,  913.52896192],
       [-545.90216334,  544.78209459],
       [-463.91159548,  465.08364153],
       [-302.69018903,  301.92985636],
       [-557.24076121,  555.47423027],
       [-135.63034291,  134.89136966],
       [-190.28006845,  191.47986546]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,160.292962,-162.052308
6,371.587526,-369.942869
7,461.154451,-459.203698
8,1822.352658,-1827.057924
9,1089.804327,-1089.564189
10,925.823191,-930.167283
11,603.380378,-603.859713
12,1112.481522,-1110.948461
13,269.260686,-269.782739
14,378.560137,-382.959731


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
